# Winequality dataset

## Feim els imports

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

from sklearn import preprocessing
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression, Perceptron, RidgeCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.feature_selection import SequentialFeatureSelector, SelectKBest, f_regression, RFE
from feature_engine.creation import MathematicalCombination, CombineWithReferenceFeature

## Preparam el dataset

En aquest apartat, unirem els dos datasets en un, després  

Funció que separa les features i les etiquetes, escalant les dades

In [ ]:
def load_data(df):
    y=df.quality
    X=df.drop('quality',axis=1)
    X = min_max_scaler.fit_transform(X)
    return X, y

Carregam els datasets a dos dataframes a partir dels fitxers csv: winequality-red.csv i winequality-white.csv

Afegim una nova columna que indiqui el tipus de vi a cada dataset i juntam els dos datasets en un de nou

In [ ]:
dfRed=pd.read_csv("winequality-red.csv", sep=";")
dfWhite=pd.read_csv("winequality-white.csv", sep=";")

dfRed['type']="Red"
dfWhite['type']="White"
df = pd.concat([dfRed,dfWhite])

Consultam les files del dataframe amb valors absents, no fa falta que eliminem cap ja que no n'hi cap amb valors absents

In [ ]:
df[df.isna().any(axis=1)]

Convertim la columna quality en una columna de valors categòrics

In [ ]:
df["quality"]=df["quality"].map(lambda x: 1 if x < 6 else 2 if x == 6 else 3)

In [ ]:
df.describe

Convertim les característiques categòriques (tipues) en númeriques

In [ ]:
labelencoder = preprocessing.LabelEncoder()

df['type']=labelencoder.fit_transform(df['type'])

Recolocam les columnes per tenir la qualitat com a ultima columna. Això no fa falta fer-ho però ho feim per a que els gràfics i altres representacions visuals quedin més entendibles

In [ ]:
df = df.reindex(columns=['fixed acidity', 'volatile acidity', 'citric acid', 
                       'residual sugar', 'chlorides', 'free sulfur dioxide',
                       'total sulfur dioxide', 'density', 'pH', 'sulphates', 
                       'alcohol', 'type', 'quality'])
df.head()

Amb el .describe() podrem veure alguns valors estadístics per a cada columna. Com la mitja, la desviació estàndard, els quartils, ...

In [ ]:
df.describe()

Amb el .info() podem veure quantes files hi ha al dataframe i el seu tipus, com veim totes les columnes consten de valors numérics

In [ ]:
df.info()

## Visualització i neteja de les dades

En aquest apartat farem diverses representacións gràfiques de les dades per poder visualitzar-les i decidir si llevam alguna mostra.

Miram quants de vins de cada qualitat hi ha

In [ ]:
df['quality'].value_counts()

Amb el boxplot podem veure el màxim i mínim no atípic, els quartils, el rang interquartil, la mitja i els outilers de cada caracterísica

In [ ]:
df.boxplot(figsize=(25,7))

Com podem veure tenim molts d'outliers, així que eliminarem els valors que estiguin per devall del percentil 0.5% i per damunt del 99.5%

Una altre manera de fer-ho és amb el següent codi que empra una fórmula basada en el rang interquartil, però trobàvem que eliminava massa files.
```python
Q1 = df.quantile(0.25)
Q3 = df.quantile(0.75)
IQR = Q3 - Q1
df = df[~((df[cols] < (Q1 - 1.5 * IQR)) |(df[cols] > (Q3 + 1.5 * IQR))).any(axis=1)]
```

In [ ]:
cols= ['fixed acidity', 'volatile acidity', 'citric acid', 
                       'residual sugar', 'chlorides', 'free sulfur dioxide',
                       'total sulfur dioxide', 'density', 'pH', 'sulphates', 
                       'alcohol']

Q01=df.quantile(0.005)
Q99=df.quantile(0.995)

print("Forma: ",df.shape)
for col in cols:
    df = df.drop(df[((df[col] < Q01[col]) | (df[col] > Q99[col]))].index)

print("Forma de després d'eliminar els outliers: ",df.shape)
df.boxplot(figsize=(25,7))


In [ ]:
df['quality'].value_counts()

Visualitzam les correlacions de les característiques entre elles amb una matriu de correlacions, això ho feim per saber quines característiques estan més o menys relacionades entre si

In [ ]:
corr = df.corr()

# Generam una màscara pel triangle superior
mask = np.triu(np.ones_like(corr, dtype=bool))

# Preparam el gràfic de pyplot
f, ax = plt.subplots(figsize=(11, 9))

# Dibuixam el mapa de calor amb la màscara i alguns paràmetres extra per millorar el resultat visual
sns.heatmap(corr, mask=mask, cmap="flare", square=True, linewidths=.25, cbar_kws={"shrink": .75})


Graficam el valor que prenen les característiques númeriques del dataframe mitjançant un histograma

In [ ]:
df.drop("type", axis=1).hist(figsize=(10,10), grid=False)

Gràfic cirular en el que podem veure la proporció de vi blanc i vermell del dataframe

In [ ]:
plt.pie(df['type'].value_counts(), labels = ["White","Red"], colors=["khaki","Maroon"])
print(df['type'].value_counts(normalize=True)*100)

## Separació del conjunt d'entrenament i el de test

Mesclam el dataframe i carregam les dades a les variables *features* (característiques) i *labels* (etiquetes). \

In [ ]:
min_max_scaler = preprocessing.MinMaxScaler()
df = df.sample(frac=1) #No faria faltar mesclar-ho ja que després els classificadors faran un shuffle internament.

features, labels = load_data(df)

Per separar les dades en els conjunts d'entrenament i de test hem emprat la funció train_test_split

Hem decidit emprar un 80% entrenament i un 20% de test ja que  amb un valor més baix d'entrenament el model de regressió logistica i el del perceptró presentaven underfiting 


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

# **Classificadors**

Per a cada model imprimim el seu classification report, la precisió de training i la de testing. A més, guardam aquests dos valors dins dos dataframes externs per després poder comparar les modificacions que farem al dataframe original.

Hem decidit fer els models dins funcions per poder cridar-les des de l'apartat d'[Enginyeria de característiques/Proves](#Proves) fora haver de reescriure el codi.

In [ ]:
dfTrainAccuracy = pd.DataFrame(columns=['Regressió Logística','Perceptró','Random Forest'])
dfTestAccuracy = pd.DataFrame(columns=['Regressió Logística','Perceptró','Random Forest'])

## Regressió Logística

Hem decidit donar-li valor al hiperparàmetre max_iters perquè amb el valor per defecte no arribava a convergir.

In [ ]:
def RegressioLogistica(df, nom):
    #Carregam les dades del dataframe i cream les variables de Train i Test
    features, labels = load_data(df)
    X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)
    
    #Cream una instancia del classificador
    logreg = LogisticRegression(max_iter=200)
    
    #Entrenam el classificador amb les dades de train
    logreg.fit(X_train,y_train)
    
    #Feim una predicció amb les dades de Test
    y_predict = logreg.predict(X_test)
    
    #Imprimim un classification report del test
    print("\nRegressio Logística:")
    print(classification_report(y_test, y_predict, zero_division=1))
    
    #Feim un score amb les dades de train i test per veure l'accuracy dels dos sets de dades
    training_accuracy = logreg.score(X_train,y_train)
    testing_accuracy = logreg.score(X_test,y_test)
    print('training accuracy:', training_accuracy*100)    
    print('testing accuracy:', testing_accuracy*100)
    
    #Afegim les dades a dos dataframes externs per després poder fer comparacions
    dfTrainAccuracy.at[nom,'Regressió Logística']=training_accuracy*100
    dfTestAccuracy.at[nom,'Regressió Logística']=testing_accuracy*100 
    
    #Cream una matriu de confusió amb les dades del test
    print("\nConfusion Matrix")
    ConfusionMatrixDisplay(confusion_matrix(y_test,y_predict, labels=logreg.classes_), display_labels=logreg.classes_).plot()
    
    plt.show()

RegressioLogistica(df, "Original")

## Perceptró

In [ ]:
def Perceptro(df, nom):
    #Carregam les dades del dataframe i cream les variables de Train i Test
    features, labels = load_data(df)
    X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)
    
    #Cream una instancia del classificador
    perceptron = Perceptron(max_iter=5000)
    
    #Entrenam el classificador amb les dades de train
    perceptron.fit(X_train,y_train)
    
    #Feim una predicció amb les dades de Test
    y_predict = perceptron.predict(X_test)
    
    #Imprimim un classification report del test
    print("\nPerceptró:")
    print(classification_report(y_test, y_predict, zero_division=1))
    
    #Feim un score amb les dades de train i test per veure l'accuracy dels dos sets de dades
    training_accuracy = perceptron.score(X_train,y_train)
    testing_accuracy = perceptron.score(X_test,y_test)
    print('training accuracy:', training_accuracy*100)    
    print('testing accuracy:', testing_accuracy*100)
    
    #Afegim les dades a dos dataframes externs per després poder fer comparacions
    dfTrainAccuracy.at[nom,'Perceptró']=training_accuracy*100
    dfTestAccuracy.at[nom,'Perceptró']=testing_accuracy*100 
    
    #Cream una matriu de confusió amb les dades del test
    print("\nConfusion Matrix")
    ConfusionMatrixDisplay(confusion_matrix(y_test,y_predict, labels=perceptron.classes_), display_labels=perceptron.classes_).plot()
    
    plt.show()

Perceptro(df, "Original")

## Random Forest

Un problema que tenim max

In [ ]:
def RandomForest(df, nom):
    #Carregam les dades del dataframe i cream les variables de Train i Test
    features, labels = load_data(df)
    X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)
    
    #Cream una instancia del classificador
    rf = RandomForestClassifier()
    
    #Entrenam el classificador amb les dades de train
    rf.fit(X_train,y_train)
    
    #Feim una predicció amb les dades de Test
    y_predict = rf.predict(X_test)
    
    #Imprimim un classification report del test
    print("\nRandom Forest:")
    print(classification_report(y_test, y_predict, zero_division=1))
    
    #Feim un score amb les dades de train i test per veure l'accuracy dels dos sets de dades
    training_accuracy = rf.score(X_train,y_train)
    testing_accuracy = rf.score(X_test,y_test)
    print('training accuracy:', training_accuracy*100)    
    print('testing accuracy:', testing_accuracy*100)
    
    #Afegim les dades a dos dataframes externs per després poder fer comparacions
    dfTrainAccuracy.at[nom,'Random Forest']=training_accuracy*100
    dfTestAccuracy.at[nom,'Random Forest']=testing_accuracy*100 
    
    #Cream una matriu de confusió amb les dades del test
    print("\nConfusion Matrix")
    ConfusionMatrixDisplay(confusion_matrix(y_test,y_predict, labels=rf.classes_), display_labels=rf.classes_).plot()
    
    plt.show()
           
RandomForest(df, "Original")

## Conclusions


El millor model és el Random Forest ja que té una precisió del 69.5% ... COMENTAR UN POC CADA CLASIFICADOR

# **Enginyeria de característiques**

L'enginyeria de característiques (feature engineering) és el procés de selecció, extracció, creació i transformació de les característiques d'un dataset amb l'objectiu de millorar l'eficàcia d'un model predictiu.

Per dur a terme l'enginyeria de característiques hem seguit els següents apartats: 
- **Feature importances dels models**: Ens permet saber la importància de cada característica dins cada model amb el fí d'eliminar les característiques menys importants.
- **Correlacions màximes i mínimes**: Ens permet saber quines característiques tenen major correlació entre elles. També ens permet saber quines característiques tenen menys correlació amb la qualitat del vi per després poder eliminar-les.
- **Feature Selection**: Selecció de les característiques més i menys importants. S'implementaran els següents:
   1. Sequential Feature Selection
   2. Univariate Feature Selection amb Kbest
   3. Recursive Feature Elimination

In [ ]:
# Hem de necessitar un dataframe fora la qualitat, ja que els métodes que empram a continuació ens
# creen una máscara de les columnes més o menys importants de longitud columnes-1. 
# Això és degut a que el nostre dataframe inclou la qualitat però els models s'entrenen fora la qualitat
# i per tant tenen una columna menys
dfWquality=df.drop("quality", axis=1)

## Feature importances de cada model

En aquest apartat emprarem els atributs dels models de classificació desenvolupats anteriorment per elegir les dues característiques amb menys importància

### Regressió logística <a id='Regressio-Logistica'></a>

In [ ]:
logreg = LogisticRegression(max_iter=5000)
logreg.fit(X_train,y_train)

logreg_odds = np.exp(logreg.coef_[0])
indices=np.argsort(logreg_odds)

fig, ax = plt.subplots()
ax.set_title("Feature importances for Logistic Regression")
ax.barh(range(len(logreg_odds)), logreg_odds[indices])
ax.set_yticks(range(len(logreg_odds)))
_ = ax.set_yticklabels(np.array(dfWquality.columns)[indices])

print("Millor característica:", dfWquality.columns[indices][-1])
pitjorLogReg = dfWquality.columns[indices][0]
print("Pitjor característica", pitjorLogReg)

### Perceptró <a id='Perceptro'></a>

In [ ]:
perceptron = Perceptron()
perceptron.fit(X_train, y_train)

perceptron_odds = np.exp(perceptron.coef_[0])
indices=np.argsort(perceptron_odds)

fig, ax = plt.subplots()
ax.set_title("Feature importances for Perceptron")
ax.barh(range(len(perceptron_odds)), perceptron_odds[indices])
ax.set_yticks(range(len(perceptron_odds)))
_ = ax.set_yticklabels(np.array(dfWquality.columns)[indices])

print("Millor característica:", dfWquality.columns[indices][-1])
pitjorPerc =dfWquality.columns[indices][0]
print("Pitjor característica:", pitjorPerc)

### Random Forest <a id='Random-Forest'></a>

In [ ]:
rf = RandomForestClassifier()
rf.fit(X_train, y_train)

importancesRF = rf.feature_importances_
indices = np.argsort(importancesRF)

fig, ax = plt.subplots()
ax.set_title("Feature importances for Random forest")
ax.barh(range(len(importancesRF)), importancesRF[indices])
ax.set_yticks(range(len(importancesRF)))
_ = ax.set_yticklabels(np.array(dfWquality.columns)[indices])

print("Millor característica:", dfWquality.columns[indices][-1])
pitjorRF =dfWquality.columns[indices][0]
print("Pitjor característica:", pitjorRF)

### Comparativa entre models
| Model | Millor característica | Pitjor característica |
| ----------- | ----------- | ----------- |
| Regressió Logística | Volatile Acidity | Alcohol |
| Perceptró | Free sulfur dioxide | Sulphates |
| Random Forest | Alcohol | Type |

Ens pareix curiós que a la regressió logística la pitjor característica sigui l'alcohol però al random forest és la més important.

Els resultats poden variar depenent de l'execució, aquests valors són els obtinguts a una execució determinada.

## Correlacions màximes i mínimes <a id='Correlacions-Minimes'></a>

In [ ]:
c = df.corr().abs()
s = c.unstack()
so = s.sort_values(kind="quicksort", ascending=False)

print("Majors Correlacions")
print(so[13:23]) #Les 12 primeres son autocorrelacions
print("-------------")
print("Menors correlacions amb qualitat")
print(so['quality'][10:13])
pitjorsCorrelacions=so['quality'][10:13].to_dict().keys()

Les característiques que tenen la major correlació amb la qualitat són el *Free sulfur dioxide* i el *Total sulfur dioxide*, així que després provarem d'unir-les \
Les característiques que tenen la menor correlació amb la qualitat són el *pH*, *residual sugars* i *sulphates*.

## [Feature Selection](https://www.scikit-learn.org/stable/modules/feature_selection.html)

El Feature Selection és un procés de selecció de característiques que ens ajudarà a reduir la quantitat de característiques del model per deixar les més importants

### [Sequential Feature Selector](https://scikit-learn.org/stable/auto_examples/feature_selection/plot_select_from_model_diabetes.html#sphx-glr-auto-examples-feature-selection-plot-select-from-model-diabetes-py) <a id='Sequential-Feature-Selector'></a>


El Sequential Feature Selector funcia amb dos procediments diferents, el Forward-SFS i el Backward-SFS. 

El Forward-SFS és un procediment greedy que troba de manera iterativa la millor característica nova per afegir al conjunt de característiques seleccionades. Inicialment començam amb zero característiques i trobam la caracteristica que maximitza una calificació de cross-validation quan un estimador es entrenat amb aquesta característica concreta. Una vegada hem afegit aquesta característica repetim es procés afegint una nova característica al conjunt de característiques seleccionades. El proces acaba quan arriba al nombre de característiques seleccionades desitjat, determinat pel paràmetre n_features_to_select.

El Backward-SFS segueix la mateixa idea, però funciona en la direcció oposada: en lloc de començar sense cap característica i afegir-ne, començam amb totes les característiques i eliminam característiques del conjunt. El paràmetre direction controla si s'utilitza el Forward-SFS o el Backward-SFS.

In [ ]:
feature_names = np.array(dfWquality.columns)
ridge = RidgeCV(alphas=np.logspace(-6, 6, num=5)).fit(X_train, y_train)

In [ ]:
sfs_forward = SequentialFeatureSelector(
    ridge, n_features_to_select=10, direction="forward"
).fit(X_train, y_train)


sfs_backward = SequentialFeatureSelector(
    ridge, n_features_to_select=10, direction="backward"
).fit(X_train, y_train)

pitjorSFSF=dfWquality.columns.difference(feature_names[sfs_forward.get_support()]).tolist()
pitjorSFSB=dfWquality.columns.difference(feature_names[sfs_backward.get_support()]).tolist()
print(type(pitjorSFSB))
print(
    "Features selected by forward sequential selection: "
    f"{feature_names[sfs_forward.get_support()]}"
    "\nFeatures not selected by forward sequential selection: "
    f"{pitjorSFSF}\n"
)

print(
    "Features selected by backward sequential selection: "
    f"{feature_names[sfs_backward.get_support()]}"
    "\nFeatures not selected by forward sequential selection: "
    f"{pitjorSFSB}"
)

### [Univariate Feature selection with Kbest](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.GenericUnivariateSelect.html#sklearn.feature_selection.GenericUnivariateSelect) <a id='Univariate-feature-selection'></a> 


El Univariate feature selection funcioan seleccionant les millors característiques basat en proves estadístiques univariades.

Nosaltres hem emprat el SelectKBest, que elimina totes les característiques excepte les k millors. No sabem molt bé com funciona internament

In [ ]:
mask = SelectKBest(f_regression, k=10).fit(X_train, y_train).get_support()

best_features = dfWquality.columns[mask].tolist()
pitjorsUFS = dfWquality.columns.difference(best_features).tolist()

print("Millors característiques: ", best_features)
print("Pitjors característiques: ", pitjorsUFS)

### [Recursive Feature Elimination](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.RFE.html#sklearn.feature_selection.RFE) <a id='Recursive-feature-elimination'></a> 


El Recursive Feature Elimination du a terme una classificació de característiques mitjançant l'eliminació recursiva de característiques.

Donat un estimador extern que assigna pesos a les característiques, l'objectiu de l'eliminació de característiques recursives (RFE) és seleccionar característiques considerant recursivament conjunts de característiques cada cop més petits. En primer lloc, l'estimador s'entrena en el conjunt inicial de característiques i la importància de cada característiques s'obté a través de qualsevol atribut . Aleshores, les característiques menys importants s'eliminen del conjunt actual de funcions. Aquest procediment es repeteix recursivament al conjunt podat fins que s'arriba al nombre desitjat de funcions per seleccionar.

In [ ]:
feature_names = np.array(dfWquality.columns)
ridge = RidgeCV(alphas=np.logspace(-6, 6, num=5)).fit(X_train, y_train)

In [ ]:
rfe_selector = RFE(ridge, n_features_to_select=10).fit(X_train, y_train)

pitjorsRFE=dfWquality.columns.difference(feature_names[rfe_selector.get_support()]).tolist()
print(
    "Característiques triades pel recursive feature elimination: "
    f"{feature_names[rfe_selector.get_support()]}"
    "\nCaracterístiques no triades pel recursive feature elimination: "
    f"{pitjorsRFE}\n"
)

### 
| Tipus | 1ra Pitjor Caract. | 2na Pitjor Caract.|
| ----------- | ----------- | ----------- |
| Sequential Feature Selection - Forward| Free sulfur dioxide | Total sulfur dioxide |
| Sequential Feature Selection - backward| Chloride | Citric Acid |
| Univariate Feature Selection | pH | Sulphates |
| Recursive Feature Elimination | Citric Acid | Type |

## 
Una vegada hem fet aquest anàlisi de les dades per arribar a la conclusió de quines eliminar o unir, podem donar lloc al següent pas, on farem modificacions al dataframe original i crearem 3 models per a cada nou dataframe. Una vegada fet això compararem les precisions de tots els models per determinar el model amb major precicisió <a id='Proves'></a>

## Eliminam característiques

En aquest subapartat eliminarem les característiques que, segons els diferents procediments fets anteriorment, tenen menor importància

In [ ]:
dfComentaris = pd.DataFrame(columns=['Comentaris'])

### Característica de menor importància de la Regressió Logística

Com s'ha observat [aquí](#Regressio-Logistica), el Feature Selection de la Regressió Logística indica que és la característica amb menys importància.

In [ ]:
df_menor_LogReg = df.drop(pitjorLogReg,axis=1)

RegressioLogistica(df_menor_LogReg, "menor imp. LogReg")
Perceptro(df_menor_LogReg, "menor imp. LogReg")
RandomForest(df_menor_LogReg, "menor imp. LogReg")

dfComentaris.at["menor imp. LogReg",'Comentaris']=pitjorLogReg

### Característica de menor importància del Perceptró

Com s'ha observat [aquí](#Perceptro), el Feature Selection del Perceptró indica que és la característica amb menys importància.

In [ ]:
df_menor_Perc = df.drop(pitjorPerc,axis=1)

RegressioLogistica(df_menor_Perc, "menor imp. Perc")
Perceptro(df_menor_Perc, "menor imp. Perc")
RandomForest(df_menor_Perc, "menor imp. Perc")

dfComentaris.at["menor imp. Perc",'Comentaris']=pitjorPerc

### Característica de menor importància del Random Forest

Com s'ha observat [aquí](#Random-Forest), el Feature Selection del Random Forest indica que és la característica amb menys importància.

In [ ]:
df_menor_RF = df.drop(pitjorRF,axis=1)

RegressioLogistica(df_menor_RF, "menor imp. RF")
Perceptro(df_menor_RF, "menor imp. RF")
RandomForest(df_menor_RF, "menor imp. RF")

dfComentaris.at["menor imp. RF",'Comentaris']=pitjorRF

### Característica amb les pitjors correlacions amb la qualitat

Com s'ha observat [aquí](#Correlacions-Minimes), són les tres característiques menys correlacionades amb la qualitat.

In [ ]:
df_Correlacions = df.drop(pitjorsCorrelacions,axis=1)

RegressioLogistica(df_Correlacions, "Pitjors carac. Correlacions")
Perceptro(df_Correlacions, "Pitjors carac. Correlacions")
RandomForest(df_Correlacions, "Pitjors carac. Correlacions")

dfComentaris.at["Pitjors carac. Correlacions",'Comentaris']=pitjorsCorrelacions

### Característica amb les pitjors característiques del Forward Feature Selection

Com s'ha observat [aquí](#Sequential-Feature-Selector), ja que el Forward Feature Selection indica que són les dues pitjors característiques.

In [ ]:
df_SFSF = df.drop(pitjorSFSF,axis=1)

RegressioLogistica(df_SFSF, "Pitjors carac. SFSF")
Perceptro(df_SFSF, "Pitjors carac. SFSF")
RandomForest(df_SFSF, "Pitjors carac. SFSF")

dfComentaris.at["Pitjors carac. SFSF",'Comentaris']=pitjorSFSF

### Pitjors característiques Backward Feature Selection

Com s'ha observat [aquí](#Sequential-Feature-Selector), el Backward Feature Selection indica que són les dues pitjors característiques.

In [ ]:
df_SFSB = df.drop(pitjorSFSB,axis=1)

RegressioLogistica(df_SFSB, "Pitjors carac. SFSB")
Perceptro(df_SFSB, "Pitjors carac. SFSB")
RandomForest(df_SFSB, "Pitjors carac. SFSB")

dfComentaris.at["Pitjors carac. SFSB",'Comentaris']=pitjorSFSB

### Pitjors característiques Univariate Feature Selection

Com s'ha observat [aquí](#Univariate-feature-selection), el SelectKBest indica que són les dues pitjors característiques.

In [ ]:
df_UFS = df.drop(pitjorsUFS,axis=1)

RegressioLogistica(df_UFS, "Pitjors carac. UFS")
Perceptro(df_UFS, "Pitjors carac. UFS")
RandomForest(df_UFS, "Pitjors carac. UFS")

dfComentaris.at["Pitjors carac. UFS",'Comentaris']=pitjorsUFS

### Pitjors característiques Recursive Feature Selection

Com hem vist [aquí](#Recursive-feature-elimination), el Recursive Feature Elimination indica que són les dues pitjors característiques.

In [ ]:
df_RFE = df.drop(pitjorsRFE,axis=1)

RegressioLogistica(df_RFE, "Pitjors carac. RFE")
Perceptro(df_RFE, "Pitjors carac. RFE")
RandomForest(df_RFE, "Pitjors carac. RFE")

dfComentaris.at["Pitjors carac. RFE",'Comentaris']=pitjorsRFE

## Cream característiques

Per crear les característiques noves hem emprat l'eina MathematicalCombinator que ens permet crear noves columnes/variables a partir d'operacions i combinacions d'altres. Per decidir quines característiques creavem o no ens hem basat en la correlació entre característiques i la nostra intuició.

### Àcids totals

Com tenim l'acidesa fixada i la volàtil, provarem de crear una de nova característica que sigui la total

In [ ]:
combinator_Acid_Tot = MathematicalCombination(
    variables_to_combine=['fixed acidity', 'volatile acidity'],
    math_operations = ['sum'],
    new_variables_names = ['total_acidity']
)

df_Acid_Tot = combinator_Acid_Tot.fit_transform(df)

RegressioLogistica(df_Acid_Tot, "Acid Totals")
Perceptro(df_Acid_Tot, "Acid Totals")
RandomForest(df_Acid_Tot, "Acid Totals")

### Percentatge d'àcid cítric respecte de l'acidesa fixada

L'àcid cítric està inclòs dins l'acidesa fixada, així que crearem una nova característica que sigui el percentatge de àcid cítric respecte la fixada.

In [ ]:
combinator_PercAC = CombineWithReferenceFeature(
    variables_to_combine=['citric acid'],
    reference_variables=['fixed acidity'],
    operations=['div'],   
    new_variables_names=['percentage_citric_acid'])

df_PercAC = combinator_PercAC.fit_transform(df)

RegressioLogistica(df_PercAC, "Perc. acid citric")
Perceptro(df_PercAC, "Perc. acid citric")
RandomForest(df_PercAC, "Perc. acid citric")

### Minerals totals

Com tenim els sulfats i els clorurs, que ambdos son minerals cream una nova caraterística anomenada minerals totals que representi la seva suma

In [ ]:
combinator_Minerals_Totals = MathematicalCombination(
    variables_to_combine=['chlorides', 'sulphates'],
    math_operations = ['sum'],
    new_variables_names = ['total_minerals']
)

df_Minerals_Totals = combinator_Minerals_Totals.fit_transform(df)

RegressioLogistica(df_Minerals_Totals, "Minerals totals")
Perceptro(df_Minerals_Totals, "Minerals totals")
RandomForest(df_Minerals_Totals, "Minerals totals")

### Percentatge de diòxid de sofre lliure

El SO2 lliure i el SO2 total son les dues característiques més relacionades així que cream una nova respecte a aquestes dues que sigui el percentatge de SO2 lliure respecte al total

In [ ]:
combinator_PercSO2 = CombineWithReferenceFeature(
    variables_to_combine=['free sulfur dioxide'],
    reference_variables=['total sulfur dioxide'],
    operations=['div'],   
    new_variables_names=['percentage_free_sulfur'])

df_PercSO2 = combinator_PercSO2.fit_transform(df)

RegressioLogistica(df_PercSO2, "Perc. SO2 lliure")
Perceptro(df_PercSO2, "Perc. SO2 lliure")
RandomForest(df_PercSO2, "Perc. SO2 lliure")

### Diòxid de sofre no lliure

Parescut a l'anterior apartat, aquí cream una nova característica que és el SO2 no lliure

In [ ]:
combinator_No_FreeSO2 = CombineWithReferenceFeature(
    variables_to_combine=['total sulfur dioxide'],
    reference_variables=['free sulfur dioxide'],
    operations=['sub'],
    new_variables_names=['non_free_sulfur_dioxide']
)

df_No_FreeSO2 = combinator_No_FreeSO2.fit_transform(df)

RegressioLogistica(df_No_FreeSO2, "S02 no lliure")
Perceptro(df_No_FreeSO2, "S02 no lliure")
RandomForest(df_No_FreeSO2, "S02 no lliure")


### Unim totes les creacions

Juntarem totes les característiques creades en un sol dataframe

In [ ]:
df_Unió = combinator_Acid_Tot.fit_transform(df)
df_Unió = combinator_Minerals_Totals.fit_transform(df_Unió)
df_Unió = combinator_PercAC.fit_transform(df_Unió)
df_Unió = combinator_No_FreeSO2.fit_transform(df_Unió)
df_Unió = combinator_PercSO2.fit_transform(df_Unió)

RegressioLogistica(df_Unió, "Totes creacions")
Perceptro(df_Unió, "Totes creacions")
RandomForest(df_Unió, "Totes creacions")

## Comparam les precisions i conclusions

En aquest apartat feim un print de les "accuracies" de tots les versions del dataframes per poder comparar-les. També calcularem la diferència de cada variació amb l'original i la difern

In [ ]:
print("Training accuracy for each model and each dataframe\n", dfTrainAccuracy)
print("\nTesting accuracy for each model and each dataframe\n", dfTestAccuracy)
print("\n\nLes features eliminades han sigut:\n", dfComentaris)

In [ ]:
dfDiffs = pd.DataFrame(columns=['Regressió Logística','Perceptró','Random Forest'])
for index, row in dfTestAccuracy.iterrows():
    dfDiffs.at[row.name]=row-dfTestAccuracy.loc['Original']
dfDiffs['Media'] = dfDiffs.mean(axis=1)
print(dfDiffs)

Com podem veure, les precisions dels models no han millorat gaire després d'aplicar els diferents processos de l'enginenyeria de característiques. Fins i tot n'hi ha colcunes que han empitjorat. 

Entre totes la que ha tengut una major millora és el dataframe: XXXX en el que hem afegit/eliminat XXXX columna. Això pot ser degut a XXXX ja que XXXX. Cal tenir en compte que els resultats poden variari d'una execució a una altre.

# **Grid Search CV**

El Grid Search és una tècnica que ens permet determinar els valors òptims dels hiperparàmetres d'un model. L'emprarem per poder millorar els nostres models. 

_(No acabam de saber que fan exactament tots els hyperparàmetres dels models, així que hem escollit un parell de cada model fora saber ben bé que fan)_

Farem feina amb el grid search damunt el dataframe: XXXX ja que com hem vist abans tenia la millor accuracy. No cream un conjunt de dades de validació ja que el grid search ho fa internament.

In [ ]:
features, labels = load_data(df)
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

## Regressió Logística

Per al grid search de la regressió lógistica mirarem els següents paràmetres:
- **Penalty**: Penalització per augmentar la magnitud dels valors dels paràmetres per reduir l'overfitting
    - 'l2': Penalització l2
    - 'l1': Penalització l1
- **C**: Inversa de la força de regularització
- **max_iter**: Nombre màxim d'iteracions que triga el solucionador en convergir

In [ ]:
param_grid={
    "C":np.logspace(-3,3,7),
    "penalty":["l1","l2"],
    "max_iter":[75,100,200,300]}

# Create a based model
logreg = LogisticRegression()
# Instantiate the grid search model
grid_search_logreg = GridSearchCV(estimator = logreg, param_grid = param_grid, cv = 10, n_jobs=-1)
grid_search_logreg.fit(X_train, y_train)

print(grid_search_logreg.best_params_)

In [ ]:
logreg = LogisticRegression()
logreg.fit(X_train, y_train)

y_predict=logreg.predict(X_test)

print("\nLogistic Regression:")
print(classification_report(y_test, y_predict,zero_division=1))

training_accuracy = logreg.score(X_train,y_train)
testing_accuracy = logreg.score(X_test,y_test)

print('training accuracy:', training_accuracy*100)    
print('testing accuracy:', testing_accuracy*100)

print("\nConfusion Matrix")
ConfusionMatrixDisplay(confusion_matrix(y_test,y_predict, labels=logreg.classes_), display_labels=logreg.classes_).plot()  
plt.show()

#---------------------------------------------------------------------------

logregTunnedParams = logreg.set_params(**grid_search_logreg.best_params_)
logregTunnedParams.fit(X_train, y_train)

y_predict=logregTunnedParams.predict(X_test)

print("\nLogistic Regression:")
print(classification_report(y_test, y_predict,zero_division=1))

training_accuracy = logregTunnedParams.score(X_train,y_train)
testing_accuracy = logregTunnedParams.score(X_test,y_test)

print('training accuracy:', training_accuracy*100)    
print('testing accuracy:', testing_accuracy*100)

print("\nConfusion Matrix")
ConfusionMatrixDisplay(confusion_matrix(y_test,y_predict, labels=logregTunnedParams.classes_), display_labels=logregTunnedParams.classes_).plot()  
plt.show()

## Perceptró

Per al grid search de la regressió lógistica mirarem els següents paràmetres:
- **Penalty**: Penalització per augmentar la magnitud dels valors dels paràmetres per reduir l'overfitting
    - 'l2': Penalització l2
    - 'l1': Penalització l1
- **Alpha**: Constant que multiplica el terme de regularització, si s'empra la regularització
- **max_iter**: Nombre màxim d'iteracions que triga el solucionador en convergir
- **early_stopping**: Si està establit a True atura l'entrenament quan el validation score deixa de millorar

In [ ]:
param_grid = {
    'penalty': ['l2', 'l1', 'elasticnet', None],
    'alpha': [0.0001, 0.001, 0.01],
    'max_iter': [75,100,1000,2500,5000],
    'early_stopping': [True, False],
}
# Create a based model
perceptro = Perceptron()
# Instantiate the grid search model
grid_search_Perc = GridSearchCV(estimator = perceptro, param_grid = param_grid, cv = 3, n_jobs = -1, verbose = 2)
grid_search_Perc.fit(X_train, y_train)

print(grid_search_Perc.best_params_)

In [ ]:
perc = Perceptron()
perc.fit(X_train, y_train)

y_predict=perc.predict(X_test)

print("PERCEPTRON ABANS DEL GRID SEARCH:")
print(classification_report(y_test, y_predict,zero_division=1))

training_accuracy = perc.score(X_train,y_train)
testing_accuracy = perc.score(X_test,y_test)

print('training accuracy:', training_accuracy*100)    
print('testing accuracy:', testing_accuracy*100)

print("\nConfusion Matrix")
ConfusionMatrixDisplay(confusion_matrix(y_test,y_predict, labels=perc.classes_), display_labels=perc.classes_).plot()  
plt.show()

#---------------------------------------------------------------------------

percTunnedParams = perc.set_params(**grid_search_Perc.best_params_)
percTunnedParams.fit(X_train, y_train)

y_predict=percTunnedParams.predict(X_test)

print("PERCEPTRON DESPRÉS DEL GRIDSEARCH:")
print(classification_report(y_test, y_predict,zero_division=1))

training_accuracy = percTunnedParams.score(X_train,y_train)
testing_accuracy = percTunnedParams.score(X_test,y_test)

print('training accuracy:', training_accuracy*100)    
print('testing accuracy:', testing_accuracy*100)

print("\nConfusion Matrix")
ConfusionMatrixDisplay(confusion_matrix(y_test,y_predict, labels=percTunnedParams.classes_), display_labels=percTunnedParams.classes_).plot()  
plt.show()

## Random Forest

Per al grid search del random forest mirarem els següents paràmetres:
- **max_depth**: Màxima profunditat d'un arbre
- **max_features**: Nombre de característiques a considerar quan cercam la millor divisió
    - 'auto': max_features=sqrt(n_features).
    - 'log2': max_features=log2(n_features).
    - 'None': max_features=n_features
- **n_estimators**: Nombre d'arbres en el bosc

In [ ]:
param_grid={
    "n_estimators":list(range(10,101,10)),
    "max_features":['auto','log2',None],
    "max_depth":[15,20,25]
    }

# Create a based model
rf = RandomForestClassifier()
# Instantiate the grid search model
grid_search_rf = GridSearchCV(estimator = rf, param_grid = param_grid, cv = 10, n_jobs = -1)
grid_search_rf.fit(X_train, y_train)

print(grid_search_rf.best_params_)


In [ ]:
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
y_predict=rf.predict(X_test)

print("\nRANDOM FOREST ABANS DEL GREEDSEARCH:")
print(classification_report(y_test, y_predict,zero_division=1))

training_accuracy = rf.score(X_train,y_train)
testing_accuracy = rf.score(X_test,y_test)

print('training accuracy:', training_accuracy*100)    
print('testing accuracy:', testing_accuracy*100)

print("\nConfusion Matrix")
ConfusionMatrixDisplay(confusion_matrix(y_test,y_predict, labels=rf.classes_), display_labels=rf.classes_).plot()  
plt.show()

#---------------------------------------------------------------------------

rfTunnedParams = rf.set_params(**grid_search_rf.best_params_)
rfTunnedParams.fit(X_train, y_train)

y_predict=rfTunnedParams.predict(X_test)

print("\nRANDOM FOREST DESPRÉS DEL GREEDSEARCH:")
print(classification_report(y_test, y_predict,zero_division=1))

training_accuracy = rfTunnedParams.score(X_train,y_train)
testing_accuracy = rfTunnedParams.score(X_test,y_test)

print('training accuracy:', training_accuracy*100)    
print('testing accuracy:', testing_accuracy*100)

print("\nConfusion Matrix")
ConfusionMatrixDisplay(confusion_matrix(y_test,y_predict, labels=rfTunnedParams.classes_), display_labels=rfTunnedParams.classes_).plot()  
plt.show()

## Resum

Com podem veure el model amb millors resultats és el Random Forest, amb una precisió de XXXX, un recall de XXXX i un F1 score de XXXX. 
El Grid Search ha calculat que els millors paràmetres pel Random Forest son Max_depth: XXXX, Max_features: XXXX, min_samples_leaf: XXXX, min_samples_split: XXXX i n_estimators: XXXX. 

Amb aquests hyperparàmetres tenim una millora del XXXX% respecte al model amb els hyperparàmetres per defecte.

# **Conclusions i resum**